<a href="https://colab.research.google.com/github/Nniikkoollaass/data-science-modul-16-HW/blob/main/DS_HW_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Вам необхідно буде побудувати веб-застосунок для візуалізації роботи вашої нейронної мережі за допомогою Streamlit або Dash. В якості домашнього завдання ви можете продовжити домашнє завдання для модуля “Згорткові нейронні мережі”.





Завдання



Створіть веб-застосунок, який дозволяє завантажувати зображення для класифікації за допомогою вашої навченої нейронної мережі.
Відобразіть вхідне зображення на веб-сторінці.
Виведіть графіки функції втрат і точності для моделі; результати класифікації (ймовірності для кожного класу та передбачений клас) у зручному форматі.
Додайте інтерфейс для вибору між двома моделями (згорткова нейромережа з Частини 1 і модель на основі VGG16 з Частини 2).




Необхідний функціонал:



- Інтерфейс для завантаження зображення, яке користувач хоче класифікувати.

- Використання завантаженої моделі для передбачення класу завантаженого зображення.

In [2]:
# Встановлення необхідних бібліотек
!pip install tensorflow streamlit opencv-python-headless matplotlib

In [5]:
# Завантаження та підготовка моделей
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

# Завантаження та підготовка датасету для першої моделі
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Завантаження та підготовка датасету для другої моделі
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = np.repeat(x_train[..., np.newaxis], 3, -1)
x_test = np.repeat(x_test[..., np.newaxis], 3, -1)
x_train = tf.image.resize(x_train, (64, 64)) / 255.0
x_test = tf.image.resize(x_test, (64, 64)) / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Створення першої моделі (звичайна згорткова нейронна мережа)
model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_cnn = model_cnn.fit(train_images, train_labels, epochs=10, batch_size=64, validation_split=0.2)
model_cnn.save('model_cnn.h5')

# Збереження історії тренування для першої моделі
np.save('history_cnn.npy', history_cnn.history)

# Створення другої моделі (модель на основі VGG16)
mobilenet_v2_base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
for layer in mobilenet_v2_base.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(mobilenet_v2_base.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)
model_vgg = Model(inputs=mobilenet_v2_base.input, outputs=output)

model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_vgg = model_vgg.fit(x_train, y_train, epochs=10, batch_size=16, validation_split=0.2)
model_vgg.save('model_vgg.h5')

# Збереження історії тренування для другої моделі
np.save('history_vgg.npy', history_vgg.history)

Epoch 1/10
750/750 [==============================] - 53s 68ms/step - loss: 0.6471 - accuracy: 0.7645 - val_loss: 0.3859 - val_accuracy: 0.8598
Epoch 2/10
750/750 [==============================] - 54s 72ms/step - loss: 0.3938 - accuracy: 0.8603 - val_loss: 0.3280 - val_accuracy: 0.8779
Epoch 3/10
750/750 [==============================] - 53s 70ms/step - loss: 0.3335 - accuracy: 0.8805 - val_loss: 0.3033 - val_accuracy: 0.8904
Epoch 4/10
750/750 [==============================] - 51s 68ms/step - loss: 0.2961 - accuracy: 0.8929 - val_loss: 0.2778 - val_accuracy: 0.8959
Epoch 5/10
750/750 [==============================] - 51s 68ms/step - loss: 0.2720 - accuracy: 0.9019 - val_loss: 0.2718 - val_accuracy: 0.9007
Epoch 6/10
750/750 [==============================] - 50s 67ms/step - loss: 0.2491 - accuracy: 0.9085 - val_loss: 0.2532 - val_accuracy: 0.9100
Epoch 7/10
750/750 [==============================] - 49s 66ms/step - loss: 0.2306 - accuracy: 0.9176 - val_loss: 0.2623 - val_accuracy:

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/10
3000/3000 [==============================] - 190s 62ms/step - loss: 0.6113 - accuracy: 0.7872 - val_loss: 0.4358 - val_accuracy: 0.8403
Epoch 2/10
3000/3000 [==============================] - 190s 63ms/step - loss: 0.4741 - accuracy: 0.8294 - val_loss: 0.4065 - val_accuracy: 0.8497
Epoch 3/10
3000/3000 [==============================] - 191s 64ms/step - loss: 0.4365 - accuracy: 0.8422 - val_loss: 0.4011 - val_accuracy: 0.8525
Epoch 4/10
3000/3000 [==============================] - 181s 60ms/step - loss: 0.4047 - accuracy: 0.8521 - val_loss: 0.3901 - val_accuracy: 0.8560
Epoch 5/10
3000/3000 [==============================] - 191s 64ms/step - loss: 0.3843 - accuracy: 0.8583 - val_loss: 0.3945 - val_accuracy: 0.8641
Epoch 6/10
3000/3000 [==============================] - 192s 64ms/step - loss: 0.3709 - accuracy: 0.8630 - val_loss: 0.3980 - val_accuracy: 0.8596
Epoch 7/10
3000/3000 [==============================] - 183s 61ms/step - loss: 0.3508 - accuracy: 0.8699 - val_loss: 0

In [7]:
# Створення веб-застосунку з використанням Streamlit
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Завантаження моделей
model_cnn = load_model('model_cnn.h5')
model_vgg = load_model('model_vgg.h5')

# Завантаження історій тренування
history_cnn = np.load('history_cnn.npy', allow_pickle=True).item()
history_vgg = np.load('history_vgg.npy', allow_pickle=True).item()

st.title('Image Classification with CNN and VGG16')

# Вибір моделі
model_option = st.selectbox('Select Model', ['CNN', 'VGG16'])

# Завантаження зображення
uploaded_file = st.file_uploader("Choose an image...", type="jpg")

if uploaded_file is not None:
    image = load_img(uploaded_file, target_size=(28, 28) if model_option == 'CNN' else (64, 64))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)

    if model_option == 'CNN':
        image = cv2.cvtColor(image[0], cv2.COLOR_BGR2GRAY).reshape((1, 28, 28, 1))
        model = model_cnn
    else:
        model = model_vgg

    st.image(uploaded_file, caption='Uploaded Image', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    # Класифікація зображення
    prediction = model.predict(image)
    st.write('Predicted Class:', np.argmax(prediction))
    st.write('Prediction Probabilities:', prediction)

    # Відображення графіків функції втрат і точності
    history = history_cnn if model_option == 'CNN' else history_vgg

    st.subheader('Model Performance')
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    ax[0].plot(history['loss'], label='Loss')
    ax[0].plot(history['val_loss'], label='Validation Loss')
    ax[0].legend()
    ax[0].set_title('Loss')

    ax[1].plot(history['accuracy'], label='Accuracy')
    ax[1].plot(history['val_accuracy'], label='Validation Accuracy')
    ax[1].legend()
    ax[1].set_title('Accuracy')

    st.pyplot(fig)

In [ ]:
#Запуск Streamlit застосунку
# Для запуску застосунку використовуємо команду:
# streamlit run app.py

Цей застосунок дозволяє завантажувати зображення, вибирати між двома моделями (CNN і VGG16), класифікувати зображення та відображати графіки функції втрат і точності для вибраної моделі.

У Google Colab ви не можете безпосередньо запустити Streamlit застосунок за допомогою команди streamlit run app.py, оскільки Colab не підтримує відображення інтерактивних веб-додатків напряму в середовищі.

Однак, є способи, які дозволяють вам запустити Streamlit-застосунок у Colab:

Використання ngrok для публікації застосунку

Встановлюмо Streamlit і ngrok:

Встановлюмо автентифікаційний токен у своєму середовищі: Відкриваємо командний рядок або термінал та вводимо команду:

In [17]:
!ngrok authtoken 2iGzBL5xv4bk2CVohD4iNKuuCPB_6RiRykwQbMdp2xeeb7Gdu

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Після цього можете повторно запустити ваш код для підключення ngrok і запуску Streamlit додатку:

Запускаємо наш Streamlit застосунок, використовуючи команду streamlit run, але на цей раз ми будемо використовувати адресу public_url:

In [18]:
import streamlit as st
from pyngrok import ngrok

# Запуск Streamlit додатку
!streamlit run app.py &

# Встановлення та налаштування ngrok
public_url = ngrok.connect(port='8501')
print(public_url)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.198.104.170:8501

  Stopping...


PyngrokNgrokHTTPError: ngrok client exception, API returned 400: {"error_code":102,"status_code":400,"msg":"invalid tunnel configuration","details":{"err":"yaml: unmarshal errors:\n  line 1: field port not found in type config.HTTPv2Tunnel"}}
